<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_function_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Providing Skills to Agents


## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install `pyautogen`:
```bash
pip install pyautogen
```

In [1]:
# %pip install "pyautogen>=0.2.3"

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [2]:
import autogen

config_list = autogen.config_list_from_json("OAI_CONFIG_LIST")

llm_config = {"config_list": config_list, "timeout": 120, "seed": 1}

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the models with matching names are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-08-01-preview',
    },
    {
        'model': 'gpt-3.5-turbo-16k',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-08-01-preview',
    },
]
```

You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/notebook/oai_openai_utils.ipynb) for full code examples of the different methods.

## Making Function Calls

In this example, we demonstrate function call execution with `AssistantAgent` and `UserProxyAgent`. With the default system prompt of `AssistantAgent`, we allow the LLM assistant to perform tasks with code, and the `UserProxyAgent` would extract code blocks from the LLM response and execute them. With the new "function_call" feature, we define functions and specify the description of the function in the OpenAI config for the `AssistantAgent`. Then we register the functions in `UserProxyAgent`.


In [3]:
def print_secret() -> str:
    """Print the secret message"""
    return "The cats name is Gaby. She is super cute but silly."


def print_ascii_cat() -> str:
    """Print an ASCII cat"""
    return r"""
    /\_/\
    >^.^<.---.
    _'-`-'     )\
    (6--\ |--\ (`.`-.
        --'  --'  ``-'
    """


def summarize_haiku(text: str) -> str:
    """A special skill to summarize a haiku"""
    return text[:10] + "..."


skill_map = {
    "print_secret": print_secret,
    "print_ascii_cat": print_ascii_cat,
    "summarize_haiku": summarize_haiku,
}

coder = autogen.AssistantAgent(
    name="coder",
    llm_config=llm_config,
    skill_map=skill_map,
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    skill_map=skill_map,
)

# start the conversation
user_proxy.initiate_chat(
    coder,
    message="Print the secret message and the ascii cat. Use this information to write a haiku and then summarize the haiku.",
)

user_proxy (to coder):

Print the secret message and the ascii cat. Use this information to write a haiku and then summarize the haiku.

--------------------------------------------------------------------------------
coder (to user_proxy):

Firstly, I will use the provided functions to print the secret message and the ASCII cat, and then create a haiku incorporating the information from the secret message. Finally, I will use the function to summarize the created haiku. Let's start by printing the secret message and the ASCII cat.

--------------------------------------------------------------------------------
user_proxy (to coder):



--------------------------------------------------------------------------------
coder (to user_proxy):

```python
# filename: print_info.py
from skills import print_secret, print_ascii_cat

def main():
    # Retrieve secret message
    secret = print_secret()
    print("Secret Message:", secret)

    # Retrieve and print ASCII cat
    ascii_cat = prin